In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('../data/external/datasets_unfiltered.csv')

In [8]:
active_df = df[df['datastore_active'] == True]

In [9]:
active_df.head(1)

,Unnamed: 0,cache_last_updated,package_id,datastore_contains_all_records_of_source_file,datastore_active,id,size,state,hash,description,...,url_type,mimetype,cache_url,name,created,url,last_modified,position,revision_id,resource_type
0,0,NaN,c6eb172c-bf35-4259-abe5-037daff76bfa,True,True,90609734-7c50-49f4-b84c-4392f408ce65,1688.0,active,NaN,Diccionario de Datos Abiertos de Ejecución Pre...,...,upload,text/csv,NaN,Diccionario de Datos,2024-03-05T13:05:11.380612,https://datos.gob.cl/dataset/c6eb172c-bf35-425...,2024-03-05T13:05:11.292814,0,c4348dd7-375b-44a9-b6af-e0c6fd140508,NaN


In [16]:
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import requests
import csv

In [17]:
def download_file(url, directory):
    try:
        response = requests.get(url)
        filename = url.split("/")[-1]
        filepath = Path.joinpath(directory, filename)
        with open(filepath, mode="wb") as file:
            file.write(response.content)
        print(f"Downloaded file {filename}")

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

In [42]:
from pathlib import Path
import pandas as pd
import requests
from urllib.parse import urlparse
import chardet
import csv
import io

def get_file_extension(url):
    return Path(urlparse(url).path).suffix.lower()

def download_file(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.content

def detect_encoding(content):
    return chardet.detect(content)['encoding']

def validate_csv(content, encoding):
    try:
        csv_reader = csv.reader(io.StringIO(content.decode(encoding)))
        # Read a few rows to check if it's valid CSV
        for _ in range(5):
            next(csv_reader)
        return True
    except Exception:
        return False

def process_url(url, output_dir):
    extension = get_file_extension(url)
    file_content = download_file(url)
    
    output_filename = output_dir / Path(urlparse(url).path).name
    
    # First, try to save the raw content
    output_filename.write_bytes(file_content)
    print(f"Raw content saved: {output_filename}")
    
    # Then, attempt to process with pandas
    try:
        match extension:
            case '.csv':
                encoding = detect_encoding(file_content)
                if validate_csv(file_content, encoding):
                    df = pd.read_csv(io.BytesIO(file_content), encoding=encoding)
                else:
                    raise ValueError("Invalid CSV format")
            case '.parquet':
                df = pd.read_parquet(io.BytesIO(file_content))
            case '.xls' | '.xlsx':
                df = pd.read_excel(io.BytesIO(file_content))
            case _:
                print(f"Unsupported file type for pandas: {extension}")
                return
        
        # If successful, save as CSV
        csv_filename = output_filename.with_suffix('.csv')
        df.to_csv(csv_filename, index=False)
        print(f"Processed and saved as CSV: {csv_filename}")
    
    except Exception as e:
        print(f"Error processing {url} with pandas: {str(e)}")
        print("File saved in raw format for manual inspection.")

In [40]:
from urllib.parse import urlparse

def get_file_extension(url):
    return Path(urlparse(url).path).suffix.lower()



def process_url(url, output_dir):
    extension = get_file_extension(url)
    
    output_filename = output_dir / Path(urlparse(url).path).name
    
    match extension:
        case '.csv':
            try: 
                dataset = pd.read_csv(url)
            except UnicodeDecodeError:
                dataset = pd.read_csv(url, encoding='latin-1', header=None)
            
            dataset.to_csv(output_filename)
        case '.xlsx':
            dataset = pd.read_excel(url)
            dataset.to_excel(output_filename)
        case _:
            raise ValueError(f'The {extension} file type is unsupported')


for index, row in active_df.iterrows():
    process_url(row.url, Path('../data/external/'))

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 12


In [43]:
active_df.url[0]

'https://datos.gob.cl/dataset/c6eb172c-bf35-4259-abe5-037daff76bfa/resource/90609734-7c50-49f4-b84c-4392f408ce65/download/diccionariodedatosabiertosinversiones.csv'